In [1]:
import pandas as pd
import numpy as np
import pickle
import warnings
warnings.filterwarnings('ignore')

print("Libraries loaded!")

Libraries loaded!


In [2]:
# Load flight data
print("Loading data...")
flights = pd.read_csv('../data/raw/flights_2019.csv', low_memory=False)

# Load saved models
with open('../src/xgb_model.pkl', 'rb') as f:
    xgb_model = pickle.load(f)

with open('../src/scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

with open('../src/features.pkl', 'rb') as f:
    feature_list = pickle.load(f)

print(f"Data loaded: {len(flights):,} flights")
print(f"Model loaded: XGBoost")
print(f"Features: {len(feature_list)}")

Loading data...
Data loaded: 4,542,343 flights
Model loaded: XGBoost
Features: 29


In [3]:
def calculate_delay_cost(delay_minutes, aircraft_type='narrow_body', passengers=150):
    """
    Calculate the cost impact of a flight delay.
    
    Parameters:
    -----------
    delay_minutes : int
        Duration of delay in minutes
    aircraft_type : str
        'narrow_body' (737, A320) or 'wide_body' (777, A350)
    passengers : int
        Number of passengers on flight
    
    Returns:
    --------
    dict : Breakdown of all costs
    """
    
    # Cost rates (industry averages)
    if aircraft_type == 'narrow_body':
        fuel_cost_per_min = 40      # $ per minute (engines idling/APU)
        crew_cost_per_min = 25      # $ per minute (pilots + flight attendants)
        maintenance_per_min = 15    # $ per minute
    else:  # wide_body
        fuel_cost_per_min = 80
        crew_cost_per_min = 45
        maintenance_per_min = 30
    
    # Passenger-related costs
    compensation_threshold = 60  # minutes before compensation kicks in
    meal_voucher = 15           # $ per passenger for delays > 2 hours
    hotel_cost = 150            # $ per passenger for overnight delays
    rebooking_cost = 50         # $ per passenger for missed connections
    
    # Calculate costs
    fuel_cost = delay_minutes * fuel_cost_per_min
    crew_cost = delay_minutes * crew_cost_per_min
    maintenance_cost = delay_minutes * maintenance_per_min
    
    # Passenger compensation
    if delay_minutes >= 180:  # 3+ hours - significant compensation
        passenger_compensation = passengers * 75
        missed_connections = int(passengers * 0.15)  # 15% miss connections
        rebooking_total = missed_connections * rebooking_cost
        meal_cost = passengers * meal_voucher
    elif delay_minutes >= 120:  # 2-3 hours
        passenger_compensation = passengers * 25
        missed_connections = int(passengers * 0.08)
        rebooking_total = missed_connections * rebooking_cost
        meal_cost = passengers * meal_voucher
    elif delay_minutes >= 60:  # 1-2 hours
        passenger_compensation = passengers * 10
        missed_connections = int(passengers * 0.05)
        rebooking_total = missed_connections * rebooking_cost
        meal_cost = 0
    else:  # Under 1 hour
        passenger_compensation = 0
        missed_connections = 0
        rebooking_total = 0
        meal_cost = 0
    
    # Customer satisfaction impact (NPS score drop estimate)
    if delay_minutes >= 180:
        nps_impact = -30
    elif delay_minutes >= 120:
        nps_impact = -20
    elif delay_minutes >= 60:
        nps_impact = -10
    elif delay_minutes >= 30:
        nps_impact = -5
    else:
        nps_impact = -2
    
    # Total cost
    total_cost = (fuel_cost + crew_cost + maintenance_cost + 
                  passenger_compensation + rebooking_total + meal_cost)
    
    return {
        'delay_minutes': delay_minutes,
        'fuel_cost': fuel_cost,
        'crew_cost': crew_cost,
        'maintenance_cost': maintenance_cost,
        'passenger_compensation': passenger_compensation,
        'missed_connections': missed_connections,
        'rebooking_cost': rebooking_total,
        'meal_cost': meal_cost,
        'total_cost': total_cost,
        'nps_impact': nps_impact,
        'cost_per_minute': total_cost / delay_minutes if delay_minutes > 0 else 0
    }

# Test the function
print("DELAY COST CALCULATOR")
print("=" * 50)

for delay in [15, 30, 60, 120, 180]:
    result = calculate_delay_cost(delay)
    print(f"\n{delay} minute delay:")
    print(f"  Total Cost: ${result['total_cost']:,.0f}")
    print(f"  NPS Impact: {result['nps_impact']} points")

DELAY COST CALCULATOR

15 minute delay:
  Total Cost: $1,200
  NPS Impact: -2 points

30 minute delay:
  Total Cost: $2,400
  NPS Impact: -5 points

60 minute delay:
  Total Cost: $6,650
  NPS Impact: -10 points

120 minute delay:
  Total Cost: $16,200
  NPS Impact: -20 points

180 minute delay:
  Total Cost: $29,000
  NPS Impact: -30 points


In [6]:
# Check what airports exist in your data
print("Sample airport codes in your data:")
print(flights['DEPARTING_AIRPORT'].unique()[:20])

print(f"\nTotal unique airports: {flights['DEPARTING_AIRPORT'].nunique()}")

# Check if our test airports exist
test_airports = ['ATL', 'ORD', 'DFW', 'JFK', 'LAX']
print("\nChecking test airports:")
for airport in test_airports:
    count = len(flights[flights['DEPARTING_AIRPORT'] == airport])
    print(f"  {airport}: {count:,} flights")

Sample airport codes in your data:
['Douglas Municipal' 'Stapleton International' 'Adams Field'
 'San Diego International Lindbergh Fl' "Chicago O'Hare International"
 'Ronald Reagan Washington National' 'Tampa International'
 'Los Angeles International' 'Pittsburgh International'
 'Honolulu International' 'Salt Lake City International'
 'Lambert-St. Louis International' 'Albany International'
 'Miami International' 'Dallas Fort Worth Regional'
 'Louis Armstrong New Orleans International' 'Kansas City International'
 'Detroit Metro Wayne County' 'Port Columbus International'
 'Portland International']

Total unique airports: 96

Checking test airports:
  ATL: 0 flights
  ORD: 0 flights
  DFW: 0 flights
  JFK: 0 flights
  LAX: 0 flights


In [7]:
def analyze_route(airport_name, flights_df):
    """
    Analyze a specific airport's performance.
    """
    
    # Filter for this airport
    airport_flights = flights_df[flights_df['DEPARTING_AIRPORT'] == airport_name].copy()
    
    if len(airport_flights) == 0:
        return {"error": f"No flights found from {airport_name}"}
    
    # Calculate metrics
    total_flights = len(airport_flights)
    delayed_flights = airport_flights['DEP_DEL15'].sum()
    delay_rate = delayed_flights / total_flights * 100
    
    # By month
    monthly_delays = airport_flights.groupby('MONTH')['DEP_DEL15'].mean() * 100
    worst_month = monthly_delays.idxmax()
    best_month = monthly_delays.idxmin()
    
    # By day of week
    daily_delays = airport_flights.groupby('DAY_OF_WEEK')['DEP_DEL15'].mean() * 100
    worst_day = daily_delays.idxmax()
    best_day = daily_delays.idxmin()
    
    # By carrier
    carrier_delays = airport_flights.groupby('CARRIER_NAME')['DEP_DEL15'].mean() * 100
    best_carrier = carrier_delays.idxmin()
    worst_carrier = carrier_delays.idxmax()
    
    # Average delay cost (assuming 45 min average delay)
    avg_delay_cost = calculate_delay_cost(45)['total_cost']
    annual_delay_cost = delayed_flights * avg_delay_cost
    
    return {
        'airport': airport_name,
        'total_flights': total_flights,
        'delayed_flights': int(delayed_flights),
        'delay_rate': delay_rate,
        'worst_month': worst_month,
        'best_month': best_month,
        'worst_day': worst_day,
        'best_day': best_day,
        'best_carrier': best_carrier,
        'worst_carrier': worst_carrier,
        'estimated_annual_delay_cost': annual_delay_cost
    }

# Test with actual airport names from your data
print("ROUTE ANALYZER")
print("=" * 60)

test_airports = [
    "Chicago O'Hare International",
    "Los Angeles International",
    "Dallas Fort Worth Regional",
    "Miami International",
    "Tampa International"
]

for airport in test_airports:
    result = analyze_route(airport, flights)
    if 'error' not in result:
        print(f"\n{airport}:")
        print(f"  Total Flights: {result['total_flights']:,}")
        print(f"  Delay Rate: {result['delay_rate']:.1f}%")
        print(f"  Best Month: {result['best_month']}")
        print(f"  Worst Month: {result['worst_month']}")
        print(f"  Best Carrier: {result['best_carrier']}")
        print(f"  Est. Annual Delay Cost: ${result['estimated_annual_delay_cost']:,.0f}")
    else:
        print(f"\n{airport}: {result['error']}")

ROUTE ANALYZER

Chicago O'Hare International:
  Total Flights: 230,460
  Delay Rate: 22.5%
  Best Month: 11
  Worst Month: 6
  Best Carrier: Comair Inc.
  Est. Annual Delay Cost: $186,336,000

Los Angeles International:
  Total Flights: 151,915
  Delay Rate: 18.9%
  Best Month: 10
  Worst Month: 6
  Best Carrier: United Air Lines Inc.
  Est. Annual Delay Cost: $103,320,000

Dallas Fort Worth Regional:
  Total Flights: 207,742
  Delay Rate: 22.2%
  Best Month: 9
  Worst Month: 6
  Best Carrier: American Eagle Airlines Inc.
  Est. Annual Delay Cost: $165,776,400

Miami International:
  Total Flights: 61,809
  Delay Rate: 18.8%
  Best Month: 11
  Worst Month: 8
  Best Carrier: Midwest Airline, Inc.
  Est. Annual Delay Cost: $41,792,400

Tampa International:
  Total Flights: 52,714
  Delay Rate: 17.9%
  Best Month: 9
  Worst Month: 6
  Best Carrier: Endeavor Air Inc.
  Est. Annual Delay Cost: $34,063,200


In [8]:
def whatif_scenario(scenario_type, current_value, new_value, flights_df):
    """
    Analyze what-if scenarios for route optimization.
    """
    
    if scenario_type == 'time_shift':
        # What if we shift flights from peak to off-peak hours?
        current_hour = current_value
        new_hour = new_value
        
        # Get delay rates by time block
        current_delay_rate = flights_df[
            flights_df['DEP_TIME_BLK'].str.startswith(str(current_hour).zfill(2))
        ]['DEP_DEL15'].mean()
        
        new_delay_rate = flights_df[
            flights_df['DEP_TIME_BLK'].str.startswith(str(new_hour).zfill(2))
        ]['DEP_DEL15'].mean()
        
        if pd.isna(current_delay_rate) or pd.isna(new_delay_rate):
            return {"error": "Invalid time values"}
        
        improvement = (current_delay_rate - new_delay_rate) * 100
        
        # Cost impact (assuming 1000 flights affected)
        flights_affected = 1000
        delays_prevented = flights_affected * (current_delay_rate - new_delay_rate)
        cost_savings = delays_prevented * calculate_delay_cost(45)['total_cost']
        
        return {
            'scenario': f"Shift flights from {current_hour}:00 to {new_hour}:00",
            'current_delay_rate': current_delay_rate * 100,
            'new_delay_rate': new_delay_rate * 100,
            'improvement_pct': improvement,
            'flights_affected': flights_affected,
            'delays_prevented': int(delays_prevented),
            'estimated_savings': cost_savings
        }
    
    elif scenario_type == 'reduce_congestion':
        # What if we reduce concurrent flights?
        current_concurrent = current_value
        target_concurrent = new_value
        
        high_congestion = flights_df[flights_df['CONCURRENT_FLIGHTS'] >= current_concurrent]['DEP_DEL15'].mean()
        low_congestion = flights_df[flights_df['CONCURRENT_FLIGHTS'] <= target_concurrent]['DEP_DEL15'].mean()
        
        improvement = (high_congestion - low_congestion) * 100
        
        return {
            'scenario': f"Reduce concurrent flights from {current_concurrent}+ to {target_concurrent}",
            'high_congestion_delay_rate': high_congestion * 100,
            'low_congestion_delay_rate': low_congestion * 100,
            'improvement_pct': improvement
        }
    
    return {"error": "Unknown scenario type"}

# Test what-if scenarios
print("WHAT-IF SCENARIO ANALYZER")
print("=" * 60)

# Scenario 1: Shift evening flights to morning
result1 = whatif_scenario('time_shift', 18, 6, flights)
print(f"\nScenario: {result1['scenario']}")
print(f"  Current Delay Rate: {result1['current_delay_rate']:.1f}%")
print(f"  New Delay Rate: {result1['new_delay_rate']:.1f}%")
print(f"  Improvement: {result1['improvement_pct']:.1f} percentage points")
print(f"  Estimated Annual Savings: ${result1['estimated_savings']:,.0f}")

# Scenario 2: Reduce congestion
result2 = whatif_scenario('reduce_congestion', 50, 30, flights)
print(f"\nScenario: {result2['scenario']}")
print(f"  High Congestion Delay Rate: {result2['high_congestion_delay_rate']:.1f}%")
print(f"  Low Congestion Delay Rate: {result2['low_congestion_delay_rate']:.1f}%")
print(f"  Improvement: {result2['improvement_pct']:.1f} percentage points")

WHAT-IF SCENARIO ANALYZER

Scenario: Shift flights from 18:00 to 6:00
  Current Delay Rate: 27.1%
  New Delay Rate: 6.9%
  Improvement: 20.2 percentage points
  Estimated Annual Savings: $727,409

Scenario: Reduce concurrent flights from 50+ to 30
  High Congestion Delay Rate: 19.5%
  Low Congestion Delay Rate: 18.8%
  Improvement: 0.6 percentage points


In [10]:
def predict_delay(month, day_of_week, carrier, airport, flights_df, model, scaler, features):
    """
    Predict if a flight will be delayed.
    """
    from sklearn.preprocessing import LabelEncoder
    
    # Get a sample row for this combination
    sample = flights_df[
        (flights_df['MONTH'] == month) &
        (flights_df['CARRIER_NAME'] == carrier) &
        (flights_df['DEPARTING_AIRPORT'] == airport)
    ]
    
    if len(sample) == 0:
        # Try with just carrier
        sample = flights_df[flights_df['CARRIER_NAME'] == carrier]
    
    if len(sample) == 0:
        return {"error": "No matching flights found"}
    
    # Get features from sample
    sample_features = sample[features].copy()
    
    # Separate numeric and categorical columns
    numeric_cols = sample_features.select_dtypes(include=['int64', 'float64']).columns.tolist()
    categorical_cols = sample_features.select_dtypes(include=['object']).columns.tolist()
    
    # Create input data - use median for numeric, mode for categorical
    input_dict = {}
    
    for col in numeric_cols:
        input_dict[col] = sample_features[col].median()
    
    for col in categorical_cols:
        input_dict[col] = sample_features[col].mode().iloc[0] if len(sample_features[col].mode()) > 0 else sample_features[col].iloc[0]
    
    input_data = pd.DataFrame([input_dict])
    
    # Update with user inputs
    input_data['MONTH'] = month
    input_data['DAY_OF_WEEK'] = day_of_week
    
    # Encode categorical columns
    for col in categorical_cols:
        le = LabelEncoder()
        le.fit(flights_df[col].astype(str))
        input_data[col] = le.transform(input_data[col].astype(str))
    
    # Ensure columns are in correct order
    input_data = input_data[features]
    
    # Scale and predict
    input_scaled = scaler.transform(input_data)
    probability = model.predict_proba(input_scaled)[0][1]
    prediction = "LIKELY DELAYED" if probability > 0.5 else "LIKELY ON-TIME"
    
    # Historical rate
    historical_rate = sample['DEP_DEL15'].mean() * 100
    
    return {
        'prediction': prediction,
        'delay_probability': probability * 100,
        'historical_delay_rate': historical_rate,
        'risk_level': 'HIGH' if probability > 0.7 else 'MEDIUM' if probability > 0.4 else 'LOW'
    }

# Test prediction
print("DELAY PREDICTOR")
print("=" * 60)

# Test a prediction
test_result = predict_delay(
    month=6,
    day_of_week=5,  # Friday
    carrier="American Airlines Inc.",
    airport="Chicago O'Hare International",
    flights_df=flights,
    model=xgb_model,
    scaler=scaler,
    features=feature_list
)

if 'error' in test_result:
    print(f"Error: {test_result['error']}")
else:
    print(f"\nTest Prediction (June, Friday, American, Chicago O'Hare):")
    print(f"  Prediction: {test_result['prediction']}")
    print(f"  Delay Probability: {test_result['delay_probability']:.1f}%")
    print(f"  Historical Rate: {test_result['historical_delay_rate']:.1f}%")
    print(f"  Risk Level: {test_result['risk_level']}")

# Test another prediction
test_result2 = predict_delay(
    month=9,
    day_of_week=2,  # Tuesday
    carrier="Delta Air Lines Inc.",
    airport="Los Angeles International",
    flights_df=flights,
    model=xgb_model,
    scaler=scaler,
    features=feature_list
)

if 'error' not in test_result2:
    print(f"\nTest Prediction (September, Tuesday, Delta, LAX):")
    print(f"  Prediction: {test_result2['prediction']}")
    print(f"  Delay Probability: {test_result2['delay_probability']:.1f}%")
    print(f"  Historical Rate: {test_result2['historical_delay_rate']:.1f}%")
    print(f"  Risk Level: {test_result2['risk_level']}")

DELAY PREDICTOR

Test Prediction (June, Friday, American, Chicago O'Hare):
  Prediction: LIKELY DELAYED
  Delay Probability: 67.7%
  Historical Rate: 32.8%
  Risk Level: MEDIUM

Test Prediction (September, Tuesday, Delta, LAX):
  Prediction: LIKELY ON-TIME
  Delay Probability: 28.6%
  Historical Rate: 15.4%
  Risk Level: LOW


In [11]:
# Save all simulator functions to a Python file
simulator_code = '''
import pandas as pd
import numpy as np
import pickle

def calculate_delay_cost(delay_minutes, aircraft_type='narrow_body', passengers=150):
    if aircraft_type == 'narrow_body':
        fuel_cost_per_min = 40
        crew_cost_per_min = 25
        maintenance_per_min = 15
    else:
        fuel_cost_per_min = 80
        crew_cost_per_min = 45
        maintenance_per_min = 30
    
    fuel_cost = delay_minutes * fuel_cost_per_min
    crew_cost = delay_minutes * crew_cost_per_min
    maintenance_cost = delay_minutes * maintenance_per_min
    
    if delay_minutes >= 180:
        passenger_compensation = passengers * 75
        missed_connections = int(passengers * 0.15)
        rebooking_total = missed_connections * 50
        meal_cost = passengers * 15
        nps_impact = -30
    elif delay_minutes >= 120:
        passenger_compensation = passengers * 25
        missed_connections = int(passengers * 0.08)
        rebooking_total = missed_connections * 50
        meal_cost = passengers * 15
        nps_impact = -20
    elif delay_minutes >= 60:
        passenger_compensation = passengers * 10
        missed_connections = int(passengers * 0.05)
        rebooking_total = missed_connections * 50
        meal_cost = 0
        nps_impact = -10
    else:
        passenger_compensation = 0
        missed_connections = 0
        rebooking_total = 0
        meal_cost = 0
        nps_impact = -2 if delay_minutes > 0 else 0
    
    total_cost = (fuel_cost + crew_cost + maintenance_cost + 
                  passenger_compensation + rebooking_total + meal_cost)
    
    return {
        "delay_minutes": delay_minutes,
        "fuel_cost": fuel_cost,
        "crew_cost": crew_cost,
        "maintenance_cost": maintenance_cost,
        "passenger_compensation": passenger_compensation,
        "missed_connections": missed_connections,
        "rebooking_cost": rebooking_total,
        "meal_cost": meal_cost,
        "total_cost": total_cost,
        "nps_impact": nps_impact
    }
'''

with open('../src/simulator.py', 'w') as f:
    f.write(simulator_code)

print("Simulator functions saved to src/simulator.py")

Simulator functions saved to src/simulator.py
